## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [2]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [3]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/1', creation_time=1765782817531, experiment_id='1', last_update_time=1765782817531, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/mlruns/0', creation_time=1765781523578, experiment_id='0', last_update_time=1765781523578, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2025/12/17 06:35:54 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.
2025/12/17 06:35:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


default artifacts URI: '/workspaces/mlops-zoomcamp/artifacts/2/55cb072839e04bd28602077f27ba7736/artifacts'
🏃 View run upset-frog-379 at: http://127.0.0.1:5000/#/experiments/2/runs/55cb072839e04bd28602077f27ba7736
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/2


In [5]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/2', creation_time=1765953354600, experiment_id='2', last_update_time=1765953354600, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/artifacts/1', creation_time=1765782817531, experiment_id='1', last_update_time=1765782817531, lifecycle_stage='active', name='nyc-taxi-experiment', tags={'mlflow.experimentKind': 'custom_model_development'}>,
 <Experiment: artifact_location='/workspaces/mlops-zoomcamp/mlruns/0', creation_time=1765781523578, experiment_id='0', last_update_time=1765781523578, lifecycle_stage='active', name='Default', tags={'mlflow.experimentKind': 'custom_model_development'}>]

### Interacting with the model registry

In [6]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [10]:
client.search_registered_models()

[<RegisteredModel: aliases={}, creation_timestamp=1765953413241, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1765953413241, latest_versions=[], name='iris-classifier', tags={}>,
 <RegisteredModel: aliases={}, creation_timestamp=1765951657115, deployment_job_id='', deployment_job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', description='', last_updated_timestamp=1765952799147, latest_versions=[<ModelVersion: aliases=[], creation_timestamp=1765951813253, current_stage='Production', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 1 was transitioned to Staging on 2025-12-17', last_updated_timestamp=1765952799147, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='f1611ab7eaac4f93b5fdd06a

In [9]:
client.search_runs(experiment_ids='1')[0].info.run_id

'17009383fb544e68bc93d4b4bf1cafc2'

In [11]:
run_id = client.search_runs(experiment_ids='2')[0].info.run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Registered model 'iris-classifier' already exists. Creating a new version of this model...
2025/12/17 06:39:51 WARNING mlflow.tracking._model_registry.fluent: Run with id 55cb072839e04bd28602077f27ba7736 has no artifacts at artifact path 'models', registering model based on models:/m-c6271b449f604a6f8dda6fc180915bba instead
2025/12/17 06:39:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1765953591221, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1765953591221, metrics=None, model_id=None, name='iris-classifier', params=None, run_id='55cb072839e04bd28602077f27ba7736', run_link='', source='models:/m-c6271b449f604a6f8dda6fc180915bba', status='READY', status_message=None, tags={}, user_id='', version='1'>